In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import calendar
import geopy.distance

In [51]:
year_on_file="2020"

## Obtener Km´s totales

In [52]:
def transform_df(ene= None):
    ene = ene.dropna(axis=0).iloc[:,0:8].reset_index().query('Fecha_Arribo != "10"').query('Hora_Retiro != "18::"').copy()
    ene["full_date_retiro"] = pd.to_datetime(ene["Fecha_Retiro"] + " " + ene["Hora_Retiro"], format="%d/%m/%Y %H:%M:%S")
    ene["full_date_arribo"] = pd.to_datetime(ene["Fecha_Arribo"] + " " + ene["Hora_Arribo"], format="%d/%m/%Y %H:%M:%S")
    ene["Mes"] = ene["full_date_retiro"].dt.month
    ene["Hora"] = ene["full_date_retiro"].dt.hour
    ene["time_delta"] = round((ene["full_date_arribo"]  - ene["full_date_retiro"]) / np.timedelta64(1,"m"),2)
    ene["Ciclo_Estacion_Retiro"]= [str(int(i)) for i in ene["Ciclo_Estacion_Retiro"].to_list()]
    ene["Bici"]= ene[["Bici"]].astype(str)
    ene["Bici"] = [i[:-2] for i in ene["Bici"]]
    ene["Ciclo_Estacion_Arribo"]= [str(int(i)) for i in ene["Ciclo_Estacion_Arribo"].to_list()]
    ene["viaje"] = ene["Ciclo_Estacion_Retiro"].astype(str)+"-"+ene["Ciclo_Estacion_Arribo"].astype(str)
    ene["Genero_Usuario"] = ene["Genero_Usuario"].fillna("X")
    ene = ene.dropna(axis=0).copy()
    return ene

def merging_files(years = None):
    years_on_file = years

    data_of_all_years = []

    for i,x in enumerate(years_on_file):
        current_year = os.listdir(f"./db/{x}")

        for m, file_on in enumerate(current_year):
            print(".",file_on[5:-4],"🤓", end=" | ")
            this_year = pd.read_csv(f"./db/{years_on_file[i]}/{file_on}", index_col=0, low_memory=False)
            year_df = transform_df(ene= this_year)
            data_of_all_years.append(year_df)

    fulldfV2 = pd.concat(data_of_all_years)
    return fulldfV2

fulldfV2 = merging_files(years = [year_on_file])

. 02_01 🤓 | . 03_01 🤓 | . 04_01 🤓 | . 05_01 🤓 | . 06_01 🤓 | . 07_01 🤓 | . 08_01 🤓 | . 09_01 🤓 | . 10_01 🤓 | . 11_01 🤓 | . dec_01 🤓 | . 01_01 🤓 | 

In [53]:
#fdv3 = transform_df(ene= pd.read_csv(f"./db/{year_on_file}/{year_on_file}_{month_on_file}_01.csv", index_col=0, low_memory=False)).replace("", np.nan).dropna(axis=0).copy()
fdv3 = fulldfV2.replace("", np.nan).dropna(axis=0).copy()

In [54]:
def estaciones_df():
   estaciones = pd.read_csv("./export/ECOBICI_Cicloestaciones.csv")
   estaciones["FID"] = [i+1 for i in estaciones["FID"].astype("int").to_list()]
   estaciones["punto_geo"] = estaciones["Y"].astype(str) +"," + estaciones["X"].astype(str)
   estaciones = estaciones.rename(columns={"FID":"id","Y":"location_lat","X":"location_lon"}).copy()
   estaciones = estaciones[["id","NOMBRE","COLONIA","location_lat","location_lon","TIPO","punto_geo"]]
   estaciones["Ciclo_Estacion_Retiro"] = estaciones["id"].astype("str").copy()
   estaciones["Ciclo_Estacion_Arribo"] = estaciones["id"].astype("str").copy()
   estaciones_retiro = estaciones.rename(columns={"NOMBRE":"name_retiro","location_lat":"location_lat_retiro","location_lon":"location_lon_retiro","punto_geo":"punto_geo_retiro"}).iloc[:,[7,1,3,4,6]].copy()
   estaciones_arribo = estaciones.rename(columns={"NOMBRE":"name_arribo","location_lat":"location_lat_arribo","location_lon":"location_lon_arribo","punto_geo":"punto_geo_arribo"}).iloc[:,[8,1,3,4,6]].copy()
   return estaciones_retiro, estaciones_arribo

estaciones_retiro, estaciones_arribo = estaciones_df()

In [55]:
def mergingfiles(month, er, ea):
   first = month.merge(er, on="Ciclo_Estacion_Retiro", how="left").merge(ea, on="Ciclo_Estacion_Arribo", how="left")
   return first

exportfileII = mergingfiles(month=fdv3, er=estaciones_retiro, ea=estaciones_arribo)

In [56]:
def get_popular_routes(dfIV = None):
    popular_routes = dfIV.copy()
    popular_routes["anio"] = popular_routes["full_date_retiro"].dt.year
    all_popular_routes = popular_routes[["anio","Mes","viaje","name_retiro","name_arribo"]].groupby(["anio","Mes","viaje","name_retiro","name_arribo"]).size().reset_index().sort_values(by=["Mes",0], ascending=False)

    month_routes = []
    
    for i in range(1,13):
        month_routes.append(all_popular_routes.query(f"Mes == {i}").head(1))

    return pd.concat(month_routes).rename(columns={0:"viajes_iniciados"})

pop_routes = get_popular_routes(dfIV=exportfileII).to_csv(f"./export/rutas_populares/{year_on_file}.csv")

In [157]:
new_routes = exportfileII[["viaje","location_lat_retiro","location_lon_retiro","location_lat_arribo","location_lon_arribo"]].copy()

def filetoexport(first):
   first = first.drop_duplicates(subset="viaje").dropna(axis=0)
        
   location_lat_retiro = first["location_lat_retiro"].to_list()
   location_lon_retiro = first["location_lon_retiro"].to_list()
   location_lat_arribo = first["location_lat_arribo"].to_list()
   location_lon_arribo = first["location_lon_arribo"].to_list()
        
   distances = pd.DataFrame({"location_dist":[geopy.distance.distance((location_lat_retiro[i],location_lon_retiro[i]), (location_lat_arribo[i],location_lon_arribo[i])).km for i in range(len(location_lon_arribo))]})
        
   l = pd.concat([first, distances], axis=1, join="inner")[["viaje","location_dist"]]
   return l

total_routes = filetoexport(first = new_routes)

In [159]:
total_routes.to_csv(f"./export/distancia_por_anio/{year_on_file}.csv")

In [160]:
listofroutes = [i for i in os.listdir("./export/rutas")]
all_routes  = pd.concat([pd.read_csv(f"./export/rutas/{x}", index_col=0) for i, x in enumerate(listofroutes)]).drop_duplicates(subset="viaje")
all_routes["location_dist"] = [round(i,2) for i in all_routes["location_dist"].to_list()]
all_routes.to_csv("./export/rutas_totales.csv")

In [166]:
def get_total_kms(datiosanio = None):
   merge_routes = pd.read_csv("./export/rutas_totales.csv", index_col=0)
   get_kms = datiosanio[["full_date_retiro","Ciclo_Estacion_Retiro","Ciclo_Estacion_Arribo"]].copy()
   get_kms["viaje"] = get_kms["Ciclo_Estacion_Retiro"]+"-"+get_kms["Ciclo_Estacion_Arribo"]
   get_kms["Dia"] = get_kms["full_date_retiro"].dt.day
   get_kms["Mes"] = get_kms["full_date_retiro"].dt.month
   get_kms["anio"] = get_kms["full_date_retiro"].dt.year
   total_kms = get_kms.merge(merge_routes, on="viaje", how="left")
   total_kms = total_kms[["Mes","Dia","anio","location_dist"]].groupby(["anio","Mes","Dia"]).sum().reset_index()
   return total_kms

year_total_kms = get_total_kms(datiosanio=fdv3)


In [167]:
year_total_kms.to_csv(f"./export/distancia_por_anio/{year_on_file}.csv")

## To Supabase

In [69]:
distancia_media =  pd.concat([pd.read_csv(f"./export/distancia_media_por_anio/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/distancia_media_por_anio"))]).query('anio >= 2019').sort_values(by=["anio","Mes","Dia"])
distancia_por_anio =  pd.concat([pd.read_csv(f"./export/distancia_por_anio/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/distancia_por_anio"))]).query('anio >= 2019').sort_values(by=["anio","Mes","Dia"])
precio_por_viajes =  pd.concat([pd.read_csv(f"./export/precio_por_viajes/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/precio_por_viajes"))]).query('anio >= 2019').sort_values(by=["anio","Mes","Dia"])
tiempo_promedio =  pd.concat([pd.read_csv(f"./export/tiempo_promedio/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/tiempo_promedio"))]).query('anio >= 2019').sort_values(by=["anio","Mes","Dia"])
viajes_por_dia =  pd.concat([pd.read_csv(f"./export/viajes_por_dia/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/viajes_por_dia"))]).query('anio >= 2019').sort_values(by=["anio","Mes","dia"])
rutas_populares = pd.concat([pd.read_csv(f"./export/rutas_populares/{x}", index_col=0) for i,x in enumerate(os.listdir("./export/viajes_por_dia"))]).query('anio >= 2019').sort_values(by=["anio","Mes"])

In [70]:
#Tiempo Promedio
tiempo_total = tiempo_promedio[["anio","Mes","time_delta"]].groupby(["anio","Mes"]).sum().reset_index().rename(columns={"time_delta":"tiempo_total"})
tiempo_promedio = tiempo_promedio[["anio","Mes","time_delta"]].groupby(["anio","Mes"]).mean().reset_index().rename(columns={"time_delta":"tiempo_promedio"})

#Viajes por mes
viajes_por_mes = viajes_por_dia[["anio","Mes","viajes"]].groupby(["anio","Mes"]).sum().reset_index()
viajes_por_mes["Mes"] = [int(i) for i in viajes_por_mes["Mes"].to_list()]

#Distance Based Tables
distancia_mediaI = distancia_media[["anio","Mes","location_dist"]].groupby(["anio","Mes"]).mean().reset_index().rename(columns={"location_dist":"distancia_promedio"})
distancia_total = distancia_por_anio[["anio","Mes","location_dist"]].groupby(["anio","Mes"]).sum().reset_index().rename(columns={"location_dist":"distancia_total"})

# Revenue Based Tables
precio_por_viajes_ptI = precio_por_viajes[["anio","Mes","precio_viaje"]].groupby(["anio","Mes"]).sum().reset_index().rename(columns={"precio_viaje":"ganancia_total"})
precio_por_viajes_ptII = precio_por_viajes[["anio","Mes","precio_viaje"]].groupby(["anio","Mes"]).mean().reset_index().rename(columns={"precio_viaje":"precio_promedio"})



In [71]:
export_fulldf = precio_por_viajes_ptI.merge(precio_por_viajes_ptII, on=["anio","Mes"], how="left").merge(distancia_mediaI, on=["anio","Mes"], how="left").merge(distancia_total, on=["anio","Mes"], how="left").merge(viajes_por_mes, on=["anio","Mes"], how="left").merge(tiempo_total, on=["anio","Mes"], how="left").merge(tiempo_promedio, on=["anio","Mes"], how="left").reset_index().set_index('index')
export_fulldfI = export_fulldf.copy().merge(rutas_populares, on=["anio","Mes"], how="left").reset_index().set_index('index')

In [72]:
export_fulldfI.to_csv("./export/supabase_df.csv")

In [73]:
export_fulldfI

,anio,Mes,ganancia_total,precio_promedio,distancia_promedio,distancia_total,viajes,tiempo_total,tiempo_promedio,viaje,name_retiro,name_arribo,viajes_iniciados
index,,,,,,,,,,,,,
0,2019,1,4200620.66,135503.892258,2.402097,731652.47,697343,337.430,10.884839,211-217,Hegel-Horacio,Hegel-Ibarbourou,390
1,2019,2,4925570.20,175913.221429,2.403036,708506.09,684092,310.605,11.093036,18-1,Reforma - Río Rhin,Río Sena-Río Balsas,413
2,2019,3,4935083.95,159196.256452,2.414677,774153.96,752236,345.650,11.150000,108-139,Nezahualcóyotl-Isabel la Católica,Mérida-Zacatecas,1
3,2019,4,4143550.82,138118.360667,2.415167,720790.55,690044,338.485,11.282833,18-1,Reforma - Río Rhin,Río Sena-Río Balsas,387
4,2019,5,4441693.43,143280.433226,2.423548,790003.22,750910,336.490,10.854516,18-1,Reforma - Río Rhin,Río Sena-Río Balsas,405
5,2019,6,4505530.00,150184.333333,2.411500,727715.43,695694,330.605,11.020167,211-217,Hegel-Horacio,Hegel-Ibarbourou,334
6,2019,7,4524350.86,145946.801935,2.414516,737332.14,703292,339.490,10.951290,211-217,Hegel-Horacio,Hegel-Ibarbourou,384
7,2019,8,4837454.81,156046.929355,2.425323,785916.65,748679,341.860,11.027742,18-1,Reforma - Río Rhin,Río Sena-Río Balsas,416
8,2019,9,4305410.83,143513.694333,2.417500,710845.58,673817,329.750,10.991667,211-217,Hegel-Horacio,Hegel-Ibarbourou,392
